In [1]:
import ROOT, os

Welcome to JupyROOT 6.30/07


In [2]:
tagger_cut = {
    'down': {'SR1': 0.8, 'SR2': 0.98},
    'up': {'SR1': 0.98, 'SR2': 2}
}

In [3]:
SR = 'SR2'
CR_cut = f"""(
    (((jet_mass>50) & (jet_mass<80)) | (jet_mass>150)) & 
    (tagger>{tagger_cut['down'][SR]}) & (tagger<{tagger_cut['up'][SR]})
)"""
SR_cut = f"""(
    (jet_mass>110) & (jet_mass<150) & 
    (tagger>{tagger_cut['down'][SR]}) & (tagger<{tagger_cut['up'][SR]})
)"""

# Read file

In [4]:
f = ROOT.TFile(f"input/Run2/background_mc.root", "r")
tree = f.Get("Events")
fit_mass = ROOT.RooRealVar("fit_mass", "fit_mass", 1500, 650, 4000)
weight = ROOT.RooRealVar("weight", "weight", 1, -10, 10)
jet_mass = ROOT.RooRealVar("jet_mass", "jet_mass", 125, 0, 999)
tagger = ROOT.RooRealVar("tagger", "tagger", 0, 0, 2)
data_region = ROOT.RooDataSet(f"data_CR", f"data_CR", tree, ROOT.RooArgSet(fit_mass, weight, jet_mass, tagger), SR_cut, "weight")


[#1] INFO:DataHandling -- RooAbsReal::attachToTree(fit_mass) TTree Float_t branch fit_mass will be converted to double precision.
[#1] INFO:DataHandling -- RooAbsReal::attachToTree(jet_mass) TTree Float_t branch jet_mass will be converted to double precision.
[#1] INFO:DataHandling -- RooAbsReal::attachToTree(tagger) TTree Float_t branch tagger will be converted to double precision.
[#1] INFO:DataHandling -- RooTreeDataStore::loadValues(data_CR) Skipping event #0 because fit_mass cannot accommodate the value 573.936
[#1] INFO:DataHandling -- RooTreeDataStore::loadValues(data_CR) Skipping event #2 because fit_mass cannot accommodate the value 520.146
[#1] INFO:DataHandling -- RooTreeDataStore::loadValues(data_CR) Skipping event #4 because fit_mass cannot accommodate the value 608.941
[#1] INFO:DataHandling -- RooTreeDataStore::loadValues(data_CR) Skipping event #8 because fit_mass cannot accommodate the value 558.591
[#1] INFO:DataHandling -- RooTreeDataStore::loadValues(data_CR) Skippi

# Fit

In [10]:
model, p1, p2, p3, result = {}, {}, {}, {}, {}

# dijet2 model
p1['dijet2'] = ROOT.RooRealVar("p1", "p1", 1, -10, 100)
p2['dijet2'] = ROOT.RooRealVar("p2", "p2", -1, -10, 10)
model['dijet2'] = ROOT.RooGenericPdf("model_background_dijet2", "model_background_dijet2", f"TMath::Power(@0, @1 + @2 * TMath::Log(@0))", ROOT.RooArgList(fit_mass, p1['dijet2'], p2['dijet2']))

# dijet3 model
p1['dijet3'] = ROOT.RooRealVar("p1", "p1", 1, -10, 100)
p2['dijet3'] = ROOT.RooRealVar("p2", "p2", -0.1, -10, 10)
p3['dijet3'] = ROOT.RooRealVar("p3", "p3", -0.1, -2, 1)
model['dijet3'] = ROOT.RooGenericPdf("model_background_dijet3", "model_background_dijet3", f"TMath::Power(@0, @1 + @2 * TMath::Log(@0) + @3 * TMath::Power(TMath::Log(@0), 2))", ROOT.RooArgList(fit_mass, p1['dijet3'], p2['dijet3'], p3['dijet3']))

# expow1 model
p1['expow1'] = ROOT.RooRealVar("p1", "p1", -0.1, -10, 0)
model['expow1'] = ROOT.RooGenericPdf("model_background_expow1", "model_background_expow1", f"TMath::Power(@0, @1)", ROOT.RooArgList(fit_mass, p1['expow1']))


# expow2 model
p1['expow2'] = ROOT.RooRealVar("p1", "p1", 0.1, -10, 5)
p2['expow2'] = ROOT.RooRealVar("p2", "p2", -0.001, -1, 0)
model['expow2'] = ROOT.RooGenericPdf("model_background_expow2", "model_background_expow2", f"TMath::Power(@0, @1) * TMath::Exp(@2 * @0)", ROOT.RooArgList(fit_mass, p1['expow2'], p2['expow2']))

# invpow2 model
p1['invpow2'] = ROOT.RooRealVar("p1", "p1", 1e-3, -1e-1, 1e-1)
p2['invpow2'] = ROOT.RooRealVar("p2", "p2", -1e2, -1e4, 1e4)
model['invpow2'] = ROOT.RooGenericPdf("model_background_invpow2", "model_background_invpow2", f"TMath::Power(1 + @1*@0, @2)", ROOT.RooArgList(fit_mass, p1['invpow2'], p2['invpow2']))

# invpow3 model
p1['invpow3'] = ROOT.RooRealVar("p1", "p1", 1e-3, -1e-1, 1e-1)
p2['invpow3'] = ROOT.RooRealVar("p2", "p2", -1e2, -1e4, 5e3)
p3['invpow3'] = ROOT.RooRealVar("p3", "p3", 1e-6, -10, 10)
model['invpow3'] = ROOT.RooGenericPdf("model_background_invpow3", "model_background_invpow3", f"TMath::Power(1 + @1*@0, @2 + @3*@0)", ROOT.RooArgList(fit_mass, p1['invpow3'], p2['invpow3'], p3['invpow3']))


for k in model:
    result[k] = model[k].fitTo(data_region, ROOT.RooFit.SumW2Error(True), Save=True)
    p1[k].setConstant(True)
    if k in p2:
        p2[k].setConstant(True)
    if k in p3:
        p3[k].setConstant(True)

[#1] INFO:NumericIntegration -- RooRealIntegral::init(model_background_dijet2_Int[fit_mass]) using numeric integrator RooRombergIntegrator to calculate Int(fit_mass)
[#1] INFO:Minimization -- RooAbsMinimizerFcn::setOptimizeConst: activating const optimization
Minuit2Minimizer: Minimize with max-calls 1000 convergence for edm < 1 strategy 1
Minuit2Minimizer : Valid minimum - status = 1
FVAL  = 211.727536053754875
Edm   = 2.47398101529286482e-07
Nfcn  = 120
p1	  = 55.6762	 +/-  10.2234	(limited)
p2	  = -4.13948	 +/-  0.731874	(limited)
[#1] INFO:Fitting -- RooAbsPdf::fitTo(model_background_dijet2) Calculating sum-of-weights-squared correction matrix for covariance matrix
[#1] INFO:Minimization -- RooAbsMinimizerFcn::setOptimizeConst: deactivating const optimization
[#1] INFO:NumericIntegration -- RooRealIntegral::init(model_background_dijet3_Int[fit_mass]) using numeric integrator RooRombergIntegrator to calculate Int(fit_mass)
[#1] INFO:Minimization -- RooAbsMinimizerFcn::setOptimizeCon

Info in <Minuit2>: MnSeedGenerator Computing seed using NumericalGradient calculator
Info in <Minuit2>: MnSeedGenerator Initial state: FCN =       305.5982572 Edm =      -92.76239791 NCalls =      9
Info in <Minuit2>: NegativeG2LineSearch Doing a NegativeG2LineSearch since one of the G2 component is negative
Info in <Minuit2>: MnSeedGenerator Negative G2 found - new state: 
  Minimum value : 214.0631533
  Edm           : 0.514791438
  Internal parameters:	[    -0.6503920185    -0.1001674212]	
  Internal gradient  :	[      64.07471599      223.7175775]	
  Internal covariance matrix:
[[  0.00023247071              0]
 [              0  2.2072947e-05]]]
Info in <Minuit2>: MnSeedGenerator Initial state  
  Minimum value : 214.0631533
  Edm           : 0.514791438
  Internal parameters:	[    -0.6503920185    -0.1001674212]	
  Internal gradient  :	[      64.07471599      223.7175775]	
  Internal covariance matrix:
[[  0.00023247071              0]
 [              0  2.2072947e-05]]]
Info in 

In [22]:
model, p1, p2, p3, result = {}, {}, {}, {}, {}

energy = 1.3e4

# dijet2 model
p1['dijet2'] = ROOT.RooRealVar("p1", "p1", 1, -50, 50)
p2['dijet2'] = ROOT.RooRealVar("p2", "p2", 1, -10, 10)
model['dijet2'] = ROOT.RooGenericPdf("model_background_dijet2", "model_background_dijet2", f"TMath::Power(@0/{energy}, @1 + @2 * TMath::Log(@0/{energy}))", ROOT.RooArgList(fit_mass, p1['dijet2'], p2['dijet2']))

# dijet3 model
p1['dijet3'] = ROOT.RooRealVar("p1", "p1", 1, -100, 100)
p2['dijet3'] = ROOT.RooRealVar("p2", "p2", 1, -100, 100)
p3['dijet3'] = ROOT.RooRealVar("p3", "p3", 1e-1, -10, 10)
model['dijet3'] = ROOT.RooGenericPdf("model_background_dijet3", "model_background_dijet3", f"TMath::Power(@0/{energy}, @1 + @2 * TMath::Log(@0/{energy}) + @3 * TMath::Power(TMath::Log(@0/{energy}), 2))", ROOT.RooArgList(fit_mass, p1['dijet3'], p2['dijet3'], p3['dijet3']))

# expow1 model
p1['expow1'] = ROOT.RooRealVar("p1", "p1", -1, -10, 10)
model['expow1'] = ROOT.RooGenericPdf("model_background_expow1", "model_background_expow1", f"TMath::Power(@0/{energy}, @1)", ROOT.RooArgList(fit_mass, p1['expow1']))

# expow2 model
p1['expow2'] = ROOT.RooRealVar("p1", "p1", -1, -100, 100)
p2['expow2'] = ROOT.RooRealVar("p2", "p2", -1, -200, 200)
model['expow2'] = ROOT.RooGenericPdf("model_background_expow2", "model_background_expow2", f"TMath::Power(@0/{energy}, @1) * TMath::Exp(@2 * @0/{energy})", ROOT.RooArgList(fit_mass, p1['expow2'], p2['expow2']))

# invpow2 model
p1['invpow2'] = ROOT.RooRealVar("p1", "p1", 1, -100, 100)
p2['invpow2'] = ROOT.RooRealVar("p2", "p2", -1, -200, 200)
model['invpow2'] = ROOT.RooGenericPdf("model_background_invpow2", "model_background_invpow2", f"TMath::Power(1 + @1*@0/{energy}, @2)", ROOT.RooArgList(fit_mass, p1['invpow2'], p2['invpow2']))

# invpow3 model
p1['invpow3'] = ROOT.RooRealVar("p1", "p1", 1, -100, 100)
p2['invpow3'] = ROOT.RooRealVar("p2", "p2", -1, -200, 200)
p3['invpow3'] = ROOT.RooRealVar("p3", "p3", 1, -100, 100)
model['invpow3'] = ROOT.RooGenericPdf("model_background_invpow3", "model_background_invpow3", f"TMath::Power(1 + @1*@0/{energy}, @2 + @3*@0/{energy})", ROOT.RooArgList(fit_mass, p1['invpow3'], p2['invpow3'], p3['invpow3']))



for k in ['dijet2', 'dijet3', 'expow1', 'expow2', 'invpow2', 'invpow3']:
    result[k] = model[k].fitTo(data_region, ROOT.RooFit.SumW2Error(True), Save=True)
    p1[k].setConstant(True)
    if k in p2:
        p2[k].setConstant(True)
    if k in p3:
        p3[k].setConstant(True)

Info in <Minuit2>: MnSeedGenerator Computing seed using NumericalGradient calculator
Info in <Minuit2>: MnSeedGenerator Initial state: FCN =       2736.757826 Edm =       33.92198126 NCalls =      9
Info in <Minuit2>: MnSeedGenerator Initial state  
  Minimum value : 2736.757826
  Edm           : 33.92198126
  Internal parameters:	[    0.02000133357     0.1001674212]	
  Internal gradient  :	[      2310.050818     -2054.692379]	
  Internal covariance matrix:
[[  1.3298285e-05              0]
 [              0  1.5331002e-05]]]
Info in <Minuit2>: VariableMetricBuilder Start iterating until Edm is < 0.001 with call limit = 1000
Info in <Minuit2>: VariableMetricBuilder    0 - FCN =       2736.757826 Edm =       33.92198126 NCalls =      9
Info in <Minuit2>: VariableMetricBuilder    1 - FCN =       2716.613262 Edm =     0.03753831435 NCalls =     15
Info in <Minuit2>: VariableMetricBuilder    2 - FCN =       2697.918013 Edm =      0.6073352973 NCalls =     27
Info in <Minuit2>: VariableMetr

In [13]:
line_color = {'expow1': ROOT.kRed, 'expow2': ROOT.kGreen+1, 'dijet2': ROOT.kYellow+2, 'dijet3': ROOT.kCyan, 'invpow2':ROOT.kBlue, 'invpow3': ROOT.kMagenta}
band_color = {'expow1': ROOT.kPink, 'expow2': ROOT.kYellow, 'dijet2': ROOT.kGreen, 'dijet3': ROOT.kCyan, 'invpow2':ROOT.kAzure, 'invpow3': ROOT.kMagenta}
candidates = ['expow1', 'expow2', 'dijet2', 'dijet3', 'invpow2', 'invpow3']

def fit_error_band(candidates, plot_name=None):
    if len(candidates) == 1:
        plot_name = candidates[0]
    else:
        plot_name = len(candidates)
    canvas = ROOT.TCanvas()

    frame = fit_mass.frame(650, 3050,24)
    data_region.plotOn(frame)

    legend = ROOT.TLegend(0.7, 0.6, 0.89, 0.89)
    #legend.SetTextSize(0.05)
    legend.SetBorderSize(0)
    legend.AddEntry(frame.getObject(0), "background MC", "lep")

    for k in candidates:
        model[k].plotOn(frame, VisualizeError=(result[k], 1), FillColor='kGray')
    for i, k in enumerate(candidates):
        model[k].plotOn(frame, LineColor=line_color[k])
        legend.AddEntry(frame.getObject(i+len(candidates)+1), k, "l")

    data_region.plotOn(frame)
    canvas.SetLogy()
    frame.SetMinimum(1e-2)
    frame.SetTitle("")
    frame.GetXaxis().SetTitle('m_{j\gamma} [GeV]')
    frame.Draw()
    legend.Draw()

    if not os.path.exists('../plots/fit/Run2'):
        os.makedirs('../plots/fit/Run2')
    canvas.SaveAs(f"../plots/fit/Run2/SRH2_{plot_name}_error_band.pdf")

In [14]:
for k in ['expow1', 'expow2', 'dijet2', 'dijet3', 'invpow2', 'invpow3']:
    fit_error_band(candidates=[k])

fit_error_band(candidates=['expow1', 'expow2', 'dijet2', 'dijet3', 'invpow2', 'invpow3'])

[#1] INFO:InputArguments -- RooAbsData::plotOn(data_CR) INFO: dataset has non-integer weights, auto-selecting SumW2 errors instead of Poisson errors
[#1] INFO:NumericIntegration -- RooRealIntegral::init(model_background_expow1_Int[fit_mass]) using numeric integrator RooRombergIntegrator to calculate Int(fit_mass)
[#1] INFO:NumericIntegration -- RooRealIntegral::init(model_background_expow1_Int[fit_mass]) using numeric integrator RooRombergIntegrator to calculate Int(fit_mass)
[#1] INFO:NumericIntegration -- RooRealIntegral::init(model_background_expow1_Int[fit_mass]) using numeric integrator RooRombergIntegrator to calculate Int(fit_mass)
[#1] INFO:NumericIntegration -- RooRealIntegral::init(model_background_expow1_Int[fit_mass]) using numeric integrator RooRombergIntegrator to calculate Int(fit_mass)
[#1] INFO:InputArguments -- RooAbsData::plotOn(data_CR) INFO: dataset has non-integer weights, auto-selecting SumW2 errors instead of Poisson errors
[#1] INFO:InputArguments -- RooAbsData

Info in <TCanvas::Print>: pdf file ../plots/fit/Run2/SRH2_expow1_error_band.pdf has been created
Info in <TCanvas::Print>: pdf file ../plots/fit/Run2/SRH2_expow2_error_band.pdf has been created
Info in <TCanvas::Print>: pdf file ../plots/fit/Run2/SRH2_dijet2_error_band.pdf has been created
Info in <TCanvas::Print>: pdf file ../plots/fit/Run2/SRH2_dijet3_error_band.pdf has been created
Info in <TCanvas::Print>: pdf file ../plots/fit/Run2/SRH2_invpow2_error_band.pdf has been created
Info in <TCanvas::Print>: pdf file ../plots/fit/Run2/SRH2_invpow3_error_band.pdf has been created
Info in <TCanvas::Print>: pdf file ../plots/fit/Run2/SRH2_6_error_band.pdf has been created
